## Data Incubator Challenge - Code

The following is the code used to perform exploratory data analysis on the data for the data incubator challenge. The project proposal relies on crawling and scraping information from multiple public sources including hospital websites, medical sites, and other public information about conditions
and where they are treated.

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from PIL import Image
from mpl_toolkits.basemap import Basemap
from geopy.geocoders import Nominatim
import math
%matplotlib inline

In [2]:
import nltk
#download stop words. to be used for filtering medical terms
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/agamino/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Stop words to filter 
stop_words = set(stopwords.words('english'))

In [4]:
#Source: https://github.com/glutanimate/wordlist-medicalterms-en
#Load medical terms, this will be use to filter out everything else
medical_terms = set(line.strip() for line in open('medical_terms.txt'))

In [5]:
#Read file with all hospital addresses and urls
#Source: Homeland Infrastructure Foundation - Level Data
#https://respond-irma-geoplatform.opendata.arcgis.com/datasets/hospitals/geoservice
df_addresses = pd.read_excel("hospital_with_addresses.xlsx")

In [6]:
#Check size
df_addresses.shape

(7030, 38)

In [7]:
#Check how many terms were loaded
len(medical_terms)

98112

In [8]:
#This file contains a list of url paths that were found on crawling
#hundreds of hospital sites.
#Will be used to extract those urls that contain medical terms
df = pd.read_excel("HospitalSites_level_name.xlsx")

In [9]:
#Check size
df.shape

(1048575, 4)

In [10]:
#Show first elements
df.head()

,domain,level,level_count,level_name
0,www.advocatechildrenshospital.com,1,35,documents
1,www.advocatechildrenshospital.com,1,15,cmc
2,www.advocatechildrenshospital.com,1,11,luth
3,www.advocatechildrenshospital.com,1,10,gsam
4,www.advocatechildrenshospital.com,1,8,ach-care-treatment


In [11]:
#Show how many links per domain
df['domain'].value_counts()

www.altabatessummit.org                 122252
www.gottliebhospital.org                109346
www.arnoldpalmerhospital.com             72525
www.advocatehealth.com                   66748
www.hopkinsmedicine.org                  34970
hospital.uillinois.edu                   27359
goodsamsanjose.com                       25818
www.kumc.edu                             20499
www.baptistjax.com                       15298
www.kentuckyonehealth.org                15107
www.ucirvinehealth.org                   13756
www.adventisthealth.org                  13724
nyulangone.org                           13695
www.chp.edu                              13476
www.akronchildrens.org                   12684
www.emmc.org                             12038
home.llu.edu                             12029
www.hoag.org                             11828
www.stmarygj.org                         10938
www.stjudemedicalcenter.org              10921
www.mercy-chicago.org                    10533
www.presenceh

In [12]:
#function to extract terms from URLs, remove extra information from urls
#and keep only medical terms
def only_medical_terms(level_name):
    #First split them into words
    level_name = str(level_name)
    results = []
    tokens = level_name.split("?")
    for t in tokens:
        s = t.split("-")
        for s1 in s:
            e = s1.split("&")
            for s2 in e:
                r = s2.split("=")
                for s3 in r:
                    if(s3 in medical_terms and s3 not in stop_words):
                        results.append(s3)
                        
    if(len(results)==0):
        results = np.nan
    return results

In [13]:
#Add a column from urls to extract medical terms
df['terms'] = df['level_name'].apply(only_medical_terms)

In [14]:
#check size
df.shape

(1048575, 5)

In [15]:
#check first few rows
df.head()

,domain,level,level_count,level_name,terms
0,www.advocatechildrenshospital.com,1,35,documents,NaN
1,www.advocatechildrenshospital.com,1,15,cmc,NaN
2,www.advocatechildrenshospital.com,1,11,luth,[luth]
3,www.advocatechildrenshospital.com,1,10,gsam,NaN
4,www.advocatechildrenshospital.com,1,8,ach-care-treatment,"[care, treatment]"


In [16]:
#Drop were no terms were found
df.dropna(inplace=True)

In [17]:
#check remaining shape
df.shape

(470814, 5)

In [18]:
#extact only the domains(urls) and terms
df_domain_terms = df[['domain','terms']]

In [ ]:
#check first few terms
df_domain_terms.head()

,domain,terms
2,www.advocatechildrenshospital.com,[luth]
4,www.advocatechildrenshospital.com,"[care, treatment]"
17,www.advocatechildrenshospital.com,"[center, fetal, care]"
19,www.advocatechildrenshospital.com,"[radiation, oncology]"
22,www.advocatechildrenshospital.com,[form]


In [ ]:
#group by domain and concatenate terms
df_grouped_by_domain = df_domain_terms.groupby('domain').apply(lambda x: x.sum())

In [ ]:
#rename index from domain to url
#remove extra 'domain' column
df_grouped_by_domain.index.names = ['url']
df.drop('domain', axis=1, inplace=True)

In [ ]:
#reset index
df_grouped_by_domain.reset_index(drop=False, inplace=True)

In [ ]:
#removed extra domain
df_grouped_by_domain.drop('domain', axis=1, inplace=True)

In [ ]:
#show first rows
df_grouped_by_domain.head()

In [ ]:
#Calculatd number of terms
df_grouped_by_domain['number_of_terms'] = df_grouped_by_domain.terms.apply(lambda x: len(x))

In [ ]:
df_grouped_by_domain.head()

In [ ]:
#Some are repeated, remove repeated terms
df_grouped_by_domain['unique_terms'] = df_grouped_by_domain.terms.apply(lambda x: set((x)))

In [ ]:
df_grouped_by_domain.head()

In [ ]:
#Add counter for number of unique terms
df_grouped_by_domain['number_of_unique'] = df_grouped_by_domain.unique_terms.apply(lambda x: len(x))

In [ ]:
df_grouped_by_domain.head()

In [ ]:
#At this point we have url, and terms ready to be merge
df_sites_terms = df_grouped_by_domain[['url','unique_terms','number_of_unique']]
df_sites_terms.columns = ['url', 'terms','number_of_terms']

In [ ]:
df_sites_terms.head()

In [ ]:
#extract only most important columns from addresses dataframe
df_addresses_cleaned = df_addresses[['url', 'NAME', 'ADDRESS', 'CITY','STATE', 'ZIP', 'POPULATION',
       'COUNTY', 'COUNTYFIPS', 'LATITUDE', 'LONGITUDE','NAICS_CODE', 'NAICS_DESC', 'SOURCE','VAL_DATE', 'WEBSITE', 'STATE_ID', 'ST_FIPS', 'OWNER',
       'BEDS']]

In [ ]:
df_addresses_cleaned.shape

In [ ]:
#drop duplicate urls (some hospitals have same URL when they have different location)
df_addresses_cleaned.drop_duplicates(['url'], keep='first', inplace=True)

In [ ]:
df_addresses_cleaned.shape

In [ ]:
df_addresses_cleaned.head()

In [ ]:
#Create one file merging terms and hospital information
#Merge terms and addresses
df_merged = pd.merge(df_sites_terms, df_addresses_cleaned, on="url")

In [ ]:
df_merged.head(10)

In [ ]:
#extract states and terms to do some calculations
df_states_terms = df_merged[['STATE','terms','number_of_terms']]

In [ ]:
df_states_terms.shape

In [ ]:
#Collect terms and group by state
state_terms = {}

In [ ]:
for index, row in df_states_terms.iterrows():
    state = row['STATE']
    terms = list(row['terms'])
    if(state in state_terms):
        state_terms[state].update(terms)
    else:
        state_terms[state] = Counter(terms)
        
        

In [ ]:
#Show California most frequent terms
state_terms['CA'].most_common(20)

In [ ]:
#create text based on frequency of top words
most_common_words_str = ""
for word in state_terms['CA'].most_common(100):
    most_common_words_str += str(word[0]+" ")*word[1]

In [ ]:
#Draw a wordcloud based on frequent terms for the state of California
#Source: http://clipart-library.com/clipart/pcodoqRRi.htm
california_outline = np.array(Image.open("state_of_california_blue.gif"))
wordcloud = WordCloud(background_color="white",width=1200, height=800,prefer_horizontal=1,mask=california_outline, random_state=42).generate(most_common_words_str)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('california_word_cloud.png')

In [ ]:
#Draw a wordcloud based on frequent terms for the state of Florida
most_commonfl_words_str = ""
for word in state_terms['FL'].most_common(100):
    most_commonfl_words_str += str(word[0]+" ")*word[1]

florida_outline = np.array(Image.open("florida_outline_v2.png"))
wordcloud = WordCloud(background_color="white",width=1200, height=800,prefer_horizontal=1,mask=florida_outline, random_state=42).generate(most_commonfl_words_str)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('florida_word_cloud.png')

In [ ]:
#Check how many types of hospitals in dataset
df_merged['NAICS_DESC'].value_counts()

In [ ]:
# Compute the correlation matrix, to see if there are any interesting ones
corr = df_merged.corr()

In [ ]:
corr

In [ ]:
#Extract state, NAICS, and terms
df_state_naics_terms = df_merged[["STATE", "NAICS_DESC","number_of_terms"]]

In [ ]:
df_state_naics_terms.head()

In [ ]:
#group by domain and concatenate terms
df_grouped_by_state = df_state_naics_terms.groupby(['STATE','NAICS_DESC']).apply(lambda x: x.sum())

In [ ]:
#rename index from domain to url
#remove extra 'domain' column
df_grouped_by_state.index.names = ['state_key','naics_key']
df_grouped_by_state.drop('STATE', axis=1, inplace=True)

In [ ]:
df_grouped_by_state.drop('NAICS_DESC', axis=1, inplace=True)

In [ ]:
df_grouped_by_state.head()

In [ ]:
#reset index
df_grouped_by_state.reset_index(drop=False, inplace=True)

In [ ]:
df_grouped_by_state.head()

In [ ]:
#Plot by state, type of hoslital (NAICS) and number of unique terms
sns.set()

# Load the pivot needed for heatmap
heatmap_data = df_grouped_by_state.pivot("state_key", "naics_key","number_of_terms")



In [ ]:
heatmap_data.fillna(0, inplace=True)
# heatmap_data.naics_key = heatmap_data.naics_key.astype(int)

In [ ]:
type(heatmap_data)

In [ ]:
# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt="f", linewidths=.5, ax=ax)

In [ ]:
df_merged.head()

In [ ]:
#Convert set items into str to search with contains
df_merged['terms_tx'] = df_merged['terms'].apply(str)

In [ ]:
#Extract those hospital where 'cancer' was found
df_hospitals_with_term_cancer = df_merged[df_merged['terms_tx'].str.contains('cancer')]

In [ ]:
df_hospitals_with_term_cancer.head()

In [ ]:
df_hospitals_with_term_cancer[['CITY','LATITUDE','LONGITUDE']]

In [ ]:
#Get longitude and latitude of cities which hospital that were
#found to have term
cities_location = []
for index, row in df_hospitals_with_term_cancer.iterrows():
    cities_location.append((row['LONGITUDE'], row['LATITUDE'],row['CITY']))

In [ ]:
#Create list of hospitals in cities that match
#the search record "Cancer" 
#Use this to determine the size of the marker on map
cities_counter = Counter()

for city in df_hospitals_with_term_cancer['CITY']:
    cities_counter.update([city])


In [ ]:
cities_counter['Chicago']

In [ ]:
#Draw map of cities with hospitals where term 'cancer' was found
scale = 5

map = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=32,lat_2=45,lon_0=-95)

# load the shapefile of the US
map.readshapefile('st99_d00', name='states', drawbounds=True)

# Get the location of each city and plot it
geolocator = Nominatim()

count = 1
for (longitude,latitude,city) in cities_location:
    x, y = map(longitude, latitude)
    #Get number of hospitals in that city 
    count = cities_counter[city]
    map.plot(x,y,marker='o',color='Green',markersize=int(math.sqrt(count))*scale)
plt.show()
plt.savefig('cities_with_cancer_facilities.png')